This notebook uses the q_learning_lib.py to perform Q-learning on tic tac toe.

A variation of different hyperparameters are tried, against many different cases.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import q_learning_lib as qlb
import json
import os

Assuming alpha = 0.1 and tau = e, lets record what happens in each of the 4 cases of training (optimal, non optimal, x, o)

In [ ]:
NUMBER_OF_BATCHES = 500
BATCH_SIZE = 10
NUMBER_OF_GAMES = NUMBER_OF_BATCHES * BATCH_SIZE

In [ ]:
def make_alpha(rate_val):
    """Function factorie for creating different alpha functions"""
    def alpha(turn):

        #return rate_val * (BATCH_SIZE * NUMBER_OF_BATCHES - turn) / (NUMBER_OF_BATCHES * BATCH_SIZE)
        #return 0.5 / (1 + rate_val * turn) 

        return 0.5 * np.exp(-rate_val * turn) #my new proposed fucntion

    return alpha

def make_tau(rate_val):
    """Function factorie for creating different tau functions"""
    def tau(turn):
        #return 1 + max_rate * (turn) / (NUMBER_OF_BATCHES * BATCH_SIZE) 
        #return 1 + (max_rate - 1) * ((turn/(NUMBER_OF_BATCHES * BATCH_SIZE))**3)
        return np.pow(10000, np.pow(turn/(NUMBER_OF_GAMES), rate_val )) #my new proposed function

    return tau


In [ ]:

def record_params(results, filename):

    df = pd.DataFrame(results[::-1], columns=['o_win', 'draw', 'x_win'])

    # Save to CSV (index=False prevents it from adding a row-number column)
    df.to_csv(filename, index=False)

In [ ]:
alpha_exponents = np.linspace(-4, -10, 10)
alpha_values = np.pow(10, alpha_exponents)
tau_exponents = np.linspace(-3, 3, 10 )
tau_values = np.pow(10, tau_exponents)
print(alpha_values)
print(tau_values)
print(tau_exponents)



In [ ]:
fig, axes = plt.subplots(figsize=(10, 10))

x = np.arange(0, len(alpha_values))

axes.plot(x, alpha_values)
#axes.plot(x, tau_values)

In [ ]:
def try_different_hyperparams(alpha_rates, tau_rates, player, strategy, folder=""):
    """this functions tries a variation of different hyperparameters
    results from every single iteration are saved in a .csv file
    how alpha params, tau values affect the final winrates is recorded in a pandas df and also saved to a file """
 
    df_list = [] #will be turned to a df later
    
    for a, alpha_rate in enumerate(alpha_rates): 
        for b, tau_rate in enumerate(tau_rates):

            #trains
            Q_Table_x_optimal, results = qlb.perform_training(player, strategy, NUMBER_OF_BATCHES, BATCH_SIZE, False, make_alpha(alpha_rate), make_tau(tau_rate))

            results = results/(BATCH_SIZE * 50)
        
            #records the resultsplayer_x_label='x', player_o_label='o', 
            filename = f"parameter_results/{player}_vs_{strategy}/_opponent_alpha_{alpha_rate}_tau_{tau_rate}.csv"
            record_params(results, filename)

            
            #records end winrates vs hyperparams
            o_winrate = np.mean(results[:, 0][-5:])
            drawrate = np.mean(results[:, 1][-5:])
            x_winrate = np.mean(results[:, 2][-5:])
            new_results = {'alpha_rate': alpha_rate, 'tau_rate': tau_rate, 'o_winrate': o_winrate, 'drawrate': drawrate, 'x_winrate': x_winrate}
            df_list.append(new_results)

            print(f"{player}_vs_{strategy}: {((a * 10 + b + 1)/(len(alpha_rates) * len(tau_values))) * 100:.2f}%")
            print(f"'alpha_rate' {alpha_rate}, 'tau_rate': {tau_rate}, 'o_winrate': {o_winrate:.3f}, 'drawrate': {drawrate:.3f}, 'x_winrate': {x_winrate:.3f}")

    df = pd.DataFrame(df_list) 
    df.to_csv(f"parameter_results/{player}_vs_{strategy}/hyperparams.csv", index=False)

In [ ]:
#try_different_hyperparams(alpha_values, tau_values, 'x', 'perfect')
#try_different_hyperparams(alpha_values, tau_values, 'o', 'perfect')
#try_different_hyperparams(alpha_values, tau_values, 'x', 'random')
#try_different_hyperparams(alpha_values, tau_values, 'o', 'random')


In [ ]:
df = pd.read_csv('parameter_results/analysis_4/x_vs_random/hyperparams.csv')

In [ ]:
# 1. Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Filter data
mask = df[np.log10(df['tau_rate']) < 1]

x = np.log10(mask['alpha_rate'])
y = np.log10(mask['tau_rate'])
z = mask['x_winrate']

# 2. Plotting
scatter = ax.scatter(x, y, c=z, s=1600, cmap='viridis', marker='s') # marker='s' often looks better for grid searches

# --- THE FORMATTING PART ---
def log_formatter(x, pos):
    return f"$10^{{{int(x)}}}$"

# Apply formatter to X axis
ax.xaxis.set_major_formatter(ticker.FuncFormatter(log_formatter))
# Apply formatter to Y axis
ax.yaxis.set_major_formatter(ticker.FuncFormatter(log_formatter))

# Ensure ticks stay at integer intervals (prevents 10^-4.5)
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
ax.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
# ---------------------------

# 3. Colorbar and Labels
cbar = fig.colorbar(scatter, ax=ax)
cbar.set_label(z.name, size=14)

# Use LaTeX for the Greek letters alpha and tau
ax.set_xlabel(r'decay rate', fontsize=14)
ax.set_ylabel(r'growth rate', fontsize=14)
ax.set_title(f"Max {z.name} achieved {z.max() * 100 :.2f}%", fontsize=16)

plt.show()

I have found good parameters for alpha rate and tau rate which should be implemented:

alpha_rate 10^-7

tau_rate = 10^-0.333


Now the goal is to create a matrix:

I will train 4 models on those parameters and record the results after playing 1000 games against each other using tau as the softmax

In [ ]:
alpha = make_alpha(np.pow(10.0, -7))
tau = make_tau(np.pow(10.0, -1/3))


In [ ]:

Q_Table_X_Perfect, results = qlb.perform_training('x', 'perfect', NUMBER_OF_BATCHES, BATCH_SIZE, False, alpha, tau)
print(results[-1, :])
Q_Table_X_Random, results = qlb.perform_training('x', 'random', NUMBER_OF_BATCHES, BATCH_SIZE, False, alpha, tau)
print(results[-1, :])
Q_Table_O_Perfect, results = qlb.perform_training('o', 'perfect', NUMBER_OF_BATCHES, BATCH_SIZE, False, alpha, tau)
print(results[-1, :])
Q_Table_O_Random, results = qlb.perform_training('o', 'random', NUMBER_OF_BATCHES, BATCH_SIZE, False, alpha, tau)
print(results[-1, :])

In [ ]:
results[-1, :]/5

In [ ]:
Q_Table_X_Random, results = qlb.perform_training('x', 'random', NUMBER_OF_BATCHES, BATCH_SIZE, True, alpha, tau, 30)
qlb.plot_training_results(results, BATCH_SIZE, NUMBER_OF_BATCHES)

In [ ]:
#Q_Table_X_Perfect["_________"]
Q_Table_X_Perfect["____o_oxx"]

In [ ]:

number_of_games_in_the_match = 10000
results_x_perfect_vs_o_perfect = qlb.Q_Table_match(Q_Table_X_Perfect, Q_Table_O_Perfect, number_of_games_in_the_match, 'softmax', 100)
#results_x_perfect_vs_o_random = qlb.Q_Table_match(Q_Table_X_Perfect, Q_Table_O_Random, number_of_games_in_the_match, 'softmax', 10)
#results_x_random_vs_o_perfect = qlb.Q_Table_match(Q_Table_X_Random, Q_Table_O_Perfect, number_of_games_in_the_match, 'softmax', 10)
#results_x_random_vs_o_random = qlb.Q_Table_match(Q_Table_X_Random, Q_Table_O_Random, number_of_games_in_the_match, 'softmax', 10)
#results_fully_random_game = qlb.Q_Table_match(Q_Table_X_Perfect, Q_Table_O_Perfect, number_of_games_in_the_match, 'softmax', 1)
#results_another_fully_random_game = qlb.play_random_match(number_of_games_in_the_match)

#results_from_matches = [results_x_perfect_vs_o_perfect, results_x_perfect_vs_o_random, results_x_random_vs_o_perfect, results_x_random_vs_o_random, results_fully_random_game, results_another_fully_random_game]

#for result_from_match in results_from_matches:
    
 #   print(result_from_match/np.sum(result_from_match))


In [ ]:
results_x_perfect_vs_o_perfect/np.sum(results_x_perfect_vs_o_perfect)

**Final results are:**

[0.2699  0.1412 0.5889]

[0.6103 0.2324 0.1573]

[0.0559 0.0836 0.8605]

[0.2667 0.3002 0.4331]


***Next stage is to compare the theoretically generated Q-Tables and the actual Q-Table that was made during learning***

In [ ]:
#comparing perfect Q-Tables

perfect_theoretical_Q_Table = qlb.import_perfect_Q_Table()

perfect_theoretical_Q_Table['____o_oxx']

for state in perfect_theoretical_Q_Table:
    
    if state.count("_") % 2 == 1:
        
        if qlb.check_result(state) == 2:

            print(state)
            print(perfect_theoretical_Q_Table[state])
            print(Q_Table_X_Perfect[state])


In [ ]:
#comparing random Q-Tables

random_theoretical_Q_Table = qlb.import_perfect_Q_Table("data_exports/random_Q_Table.json")

for state in random_theoretical_Q_Table:
    
    if state.count("_") % 2 == 1:
        
        if qlb.check_result(state) == 2:

            print(state)
            print(random_theoretical_Q_Table[state])
            print(Q_Table_X_Random[state])

In [ ]:
perfect_theoretical_Q_Table['____o_oxx']

Good results, I have implemented that into my report. Now I will generate some graphs for report


In [ ]:
#No Learning over 1000 batches of 10 (hyperparameter tau = 1, alpha = 0, means that )

qlb.plot_training_results(results, BATCH_SIZE, NUMBER_OF_BATCHES)

In [ ]:
results

In [ ]:
results[:, 0]/(366+134) < 0.05

In [ ]:
# Train via Q-Learning (Existing)
q_table_td, results_td = qlb.perform_training('x', opponent_type='random', number_of_batches=NUMBER_OF_BATCHES, batch_size=BATCH_SIZE)

# Train via Monte Carlo (New)
q_table_mc, results_mc = qlb.perform_training_MC_incremental('x', opponent_type='random')


In [ ]:

# Plot and compare
qlb.plot_training_results(results_td, BATCH_SIZE, NUMBER_OF_BATCHES)
qlb.plot_training_results(results_mc, BATCH_SIZE, NUMBER_OF_BATCHES)